In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# https://arxiv.org/pdf/1811.04670.pdf -> trying to implement this
import pandas as pd

import utils
import nnlm_eager as nnlm
import models
import train
import time
import os

from tensorflow.keras import optimizers

import tensorflow as tf
tf.enable_eager_execution()

Using TensorFlow backend.
W0809 20:18:55.795169 139740363224896 deprecation_wrapper.py:119] From /home/gowtham/Desktop/Learning/fake_news/utils.py:6: The name tf.enable_eager_execution is deprecated. Please use tf.compat.v1.enable_eager_execution instead.



In [3]:
glove_dict = utils.prepare_glove_dict('pretrained_embeddings/glove/glove.6B.300d.txt')

400000it [00:32, 12158.12it/s]


In [4]:
# read dataframes
train_df = pd.read_csv('LIAR-PLUS/dataset/train2.tsv', sep='\t', header=None)
val_df = pd.read_csv('LIAR-PLUS/dataset/val2.tsv', sep='\t', header=None)
test_df = pd.read_csv('LIAR-PLUS/dataset/test2.tsv', sep='\t', header=None)
# drop the first column which is just the indexes
train_df.drop([0], axis=1, inplace=True)
val_df.drop([0], axis=1, inplace=True)
test_df.drop([0], axis=1, inplace=True)
# change column names
column_names = ['id', 'label', 'statement', 'subjects', 'speaker',
                'speaker_job_title', 'state', 'party',
                'barely_true_counts', 'false_counts', 'half_true_counts',
                'mostly_true_counts', 'pants_fire_counts', 'context',
                'justification']

train_df.columns = column_names
val_df.columns = column_names
test_df.columns = column_names
# split labels for binarising 
unique_labels = set(train_df['label'].unique())
true_labels = set(['true', 'mostly-true', 'half-true'])
false_labels = set.difference(unique_labels, true_labels)

print(unique_labels, true_labels, false_labels)
def binarise_label(label):
    if label in true_labels:
        return True
    else:
        return False
updated_column_names = column_names[:2] + ['binary_label'] + column_names[2:]

train_df['binary_label'] = train_df['label'].map(binarise_label)
val_df['binary_label'] = val_df['label'].map(binarise_label)
test_df['binary_label'] = test_df['label'].map(binarise_label)

train_df = train_df.reindex(columns=updated_column_names)
val_df = val_df.reindex(columns=updated_column_names)
test_df = test_df.reindex(columns=updated_column_names)


print('train_df shape  :',train_df.shape)
print('val_df shape    :',val_df.shape)
print('test_df shape   :',test_df.shape)

{'false', 'half-true', 'mostly-true', 'pants-fire', 'barely-true', 'true'} {'true', 'mostly-true', 'half-true'} {'barely-true', 'pants-fire', 'false'}
train_df shape  : (10240, 16)
val_df shape    : (1284, 16)
test_df shape   : (1267, 16)


In [5]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(train_df['label'].values)
train_df['label_encoded'] = le.transform(train_df['label'].values)
val_df['label_encoded'] = le.transform(val_df['label'].values)
test_df['label_encoded'] = le.transform(test_df['label'].values)


# lets merge all the dfs for eda on the whole set
merged_df = train_df.append(val_df).append(test_df)

merged_df = merged_df.apply(lambda x: x.str.lower() if x.dtype == "object" else x)
train_df = train_df.apply(lambda x: x.str.lower() if x.dtype == "object" else x) 
val_df = val_df.apply(lambda x: x.str.lower() if x.dtype == "object" else x)
test_df = test_df.apply(lambda x: x.str.lower() if x.dtype == "object" else x) 

In [6]:
# TRAIN_CSV_PATH = 'LIAR-PLUS/dataset/train2.csv'
# VAL_CSV_PATH = 'LIAR-PLUS/dataset/val2.csv'
# TEST_CSV_PATH = 'LIAR-PLUS/dataset/test2.csv'

# EMBEDDING_DIR = 'pretrained_embeddings/glove/'


MAX_LEN_STATMENT = 50
MAX_LEN_JUSTIFICATION = 100

LSTM_HIDDEN_UNITS_STATEMENT = 256
LSTM_HIDDEN_UNITS_JUSTIFICATION = 256

META_MODEL_UNITS = 256
COUNTS_MODEL_UNITS = 64
MULTIMODE_MODEL_UNITS = 1024

EMBEDDING_DIM = 128

SENTENCE_ENCODER = 'GLOVE'

BINARY_CLASSIFICATION = False

if BINARY_CLASSIFICATION:
    NUM_CLASSES = 2
    CHECKPOINT_PATH = 'checkpoints_binary/'
else:
    NUM_CLASSES = 6
    CHECKPOINT_PATH = 'checkpoints_multiclass/'

NUM_EPOCHS = 20
BATCH_SIZE = 32
LEARNING_RATE = 0.001

TRAIN_TEXT_ENCODER = False
REPEAT_FIRST_BATCH = False


In [7]:
merged_df.head()

,id,label,binary_label,statement,subjects,speaker,speaker_job_title,state,party,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_fire_counts,context,justification,label_encoded
0,2635.json,false,False,says the annies list political group supports ...,abortion,dwayne-bohac,state representative,texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,that's a premise that he fails to back up. ann...,1
1,10540.json,half-true,True,when did the decline of coal start? it started...,"energy,history,job-accomplishments",scott-surovell,state delegate,virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,"surovell said the decline of coal ""started whe...",2
2,324.json,mostly-true,True,"hillary clinton agrees with john mccain ""by vo...",foreign-policy,barack-obama,president,illinois,democrat,70.0,71.0,160.0,163.0,9.0,denver,obama said he would have voted against the ame...,3
3,1123.json,false,False,health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,the release may have a point that mikulskis co...,1
4,9028.json,half-true,True,the economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on cnn,"crist said that the economic ""turnaround start...",2


((10240, 17), (1284, 17), (1267, 17))

In [22]:
# encoding text columns for BILSTM
statements_tf, statement_utils, statement_emb_matrix = utils.prepare_text_column(
    merged_df, 'statement', 50, glove_dict)
jobs_tf, jobs_utils, jobs_emb_matrix = utils.prepare_text_column(
    merged_df, 'speaker_job_title', 20, glove_dict)
contexts_tf, contexts_utils, contexts_emb_matrix = utils.prepare_text_column(
    merged_df, 'context', 25, glove_dict)
subjects_tf, subjects_utils, subjects_emb_matrix = utils.prepare_text_column(
    merged_df, 'subjects', 5, glove_dict)

vocab_size for statement is 10001


vocab_size for speaker_job_title is 1375


vocab_size for context is 3303


vocab_size for subjects is 4536


In [ ]:
train_size = train_df.shape[0]
train_val_size = train_df.shape[0] + val_df.shape[0]

train_statements, val_statements, test_statements = statements_tf[:train_size], 
                                                    statements_tf[train_size:train_val_size],
                                                    statements_tf[train_val_size:]
train_jobs, val_jobs, test_jobs = jobs_tf[:train_size], 
                                  jobs_tf[train_size:train_val_size],
                                  jobs_tf[train_val_size:]
train_contexts, val_contexts, test_contexts = contexts_tf[:train_size], 
                                              contexts_tf[train_size:train_val_size],
                                              contexts_tf[train_val_size:]
train_subjects, val_subjects, test_subjects = subjects_tf[:train_size], 
                                              subjects_tf[train_size:train_val_size],
                                              subjects_tf[train_val_size:]